In [20]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import tempfile


In [21]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)


In [22]:
words = []
classes = []
documents = []
ignore_words = []

for intent in intents ['intents']:
    for pattern in intent ['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

            words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
            words = sorted(list(set(words)))
            classes = sorted(list(set(classes)))
            print(len(documents), "documents")
            print (len(classes), "classes", classes)
            print (len(words), "unique stemmed words", words)

            pickle.dump(words, open('words.pkl', 'wb'))
            pickle.dump(classes, open('classes.pkl', 'wb'))




1 documents
1 classes ['unknown']
1 unique stemmed words ['jbjkbk']
7 documents
2 classes ['greeting', 'unknown']
6 unique stemmed words ['126889', 'hi', 'jbjkbk', 'kdd', 'kjbh', 'ljhh']
13 documents
3 classes ['goodbye', 'greeting', 'unknown']
19 unique stemmed words ['126889', '?', 'anyon', 'ar', 'cya', 'day', 'good', 'hello', 'hi', 'how', 'is', 'jbjkbk', 'kdd', 'kjbh', 'ljhh', 'ther', 'up', 'what', 'you']
17 documents
4 classes ['age', 'goodbye', 'greeting', 'unknown']
25 unique stemmed words ['126889', '?', 'a', 'anyon', 'ar', 'cya', 'day', 'good', 'goodby', 'hav', 'hello', 'hi', 'how', 'is', 'jbjkbk', 'kdd', 'kjbh', 'lat', 'ljhh', 'old', 'see', 'ther', 'up', 'what', 'you']
22 documents
5 classes ['age', 'goodbye', 'greeting', 'name', 'unknown']
29 unique stemmed words ['126889', '?', 'a', 'ag', 'anyon', 'ar', 'cya', 'day', 'good', 'goodby', 'hav', 'hello', 'hi', 'how', 'is', 'jbjkbk', 'kdd', 'kjbh', 'lat', 'ljhh', 'nam', 'old', 'see', 'ther', 'tim', 'up', 'what', 'yo', 'you']
25 d

In [23]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])
print ("Training data created")



Training data created
ipykernel_launcher:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [24]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape=(len (train_x[0]),), activation ='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(train_y[0]), activation='softmax'))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               12672     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 22)                1430      
Total params: 22,358
Trainable params: 22,358
Non-trainable params: 0
_________________________________________________________________


In [26]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [27]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)

Epoch 1/50
2509/2509 [==============================] - 10s 4ms/step - loss: 0.5656 - accuracy: 0.8396
Epoch 2/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0887 - accuracy: 0.9736
Epoch 3/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0584 - accuracy: 0.9798
Epoch 4/50
2509/2509 [==============================] - 9s 4ms/step - loss: 0.0496 - accuracy: 0.9821
Epoch 5/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0411 - accuracy: 0.9837
Epoch 6/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0385 - accuracy: 0.9844
Epoch 7/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0342 - accuracy: 0.9864
Epoch 8/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.0360 - accuracy: 0.9853
Epoch 9/50
2509/2509 [==============================] - 9s 3ms/step - loss: 0.0331 - accuracy: 0.9865
Epoch 10/50
2509/2509 [==============================] - 8s 3ms/step - loss: 0.03

In [28]:
def clean_up_sentence (sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
            

In [29]:
p = bow("How are you today", words)
print (p)
print (classes)

found in bag: how
found in bag: ar
found in bag: you
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
['Hate', 'None', 'age', 'banter', 'body-pain', 'breathing', 'cough', 'diarrhoea', 'fever', 'goodbye', 'greeting', 'headache', 'name', 'nasal', 'noanswer', 'options', 'sore-throat', 'thanks', 'tiredness', 'unknown', 'vomit', 'yes']


In [30]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])
print(model.predict(inputvar))

[[4.66663947e-12 2.73696013e-14 1.60234482e-11 1.16554275e-15
  2.74492462e-16 2.15597740e-18 7.46677145e-17 1.01312684e-17
  2.44866609e-20 3.29496223e-14 1.00000000e+00 9.45769278e-18
  9.99029201e-13 6.77297878e-15 3.08731037e-15 2.85500551e-12
  8.85770190e-12 1.18801421e-15 4.34665068e-16 3.53833429e-16
  1.16727754e-12 4.83941670e-14]]


In [31]:
 model.save("chappy-bot-model.h5", hist)



In [32]:


pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open ( "chappy-bot-data.pkl", "wb"))

